# Imports

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from google.colab import drive
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Input, MaxPooling2D, GlobalAveragePooling2D, Dense, MaxPool2D
from tensorflow.keras.utils import to_categorical
import keras.backend as K
from keras.layers import ReLU, GlobalAvgPool2D, Concatenate, AvgPool2D
from PIL import Image

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Lidando com as Imagens

In [ ]:
imagesTesting = []
labelsTesting = []
imagesTraining = []
labelsTraining = []
classLabels = ['barberry', 'bayberry', 'beach plum', 'bearberry', 'black berry', 'black cherry', 'blueberry', 'ceylon gooseberry', 'chokeberry', 'crowberry']

target_size = (224, 224)

def readTestingImages():
    os.chdir('/content/drive/MyDrive/CNN_SOURCE/Testing/Testing')

    imageCountList = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for filename in os.listdir():
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Read Image
            image = cv2.imread(filename)

            # Resize the image
            image_resized = cv2.resize(image, target_size)

            # Normalize the image
            image_normalized = image_resized / 255.0

            # Extract label from filename
            labels = filename.rsplit("_", 1)
            label = labels[0].replace('_', ' ')

            # Get the index of the label in classLabels
            label_index = classLabels.index(label)
            imageCountList[label_index] += 1

            if imageCountList[label_index] > 20:
              continue

            imagesTesting.append(image_normalized)
            labelsTesting.append(label_index)

def readTrainingImages():
    os.chdir('/content/drive/MyDrive/CNN_SOURCE/Training/Training')

    imageCountList = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for filename in os.listdir():
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Read Image
            image = cv2.imread(filename)

            # Resize the image
            image_resized = cv2.resize(image, target_size)

            # Normalize the image
            image_normalized = image_resized / 255.0

            # Extract label from filename
            labels = filename.rsplit("_", 1)
            label = labels[0].replace('_', ' ')

            # Get the index of the label in classLabels
            label_index = classLabels.index(label)
            imageCountList[label_index] += 1

            if imageCountList[label_index] > 80:
              continue

            imagesTraining.append(image_normalized)
            labelsTraining.append(label_index)

readTestingImages()
readTrainingImages()

# Arquitetura Densenet

In [ ]:
def densenet(img_shape, n_classes, f=32):
  repetitions = 6, 12, 24, 16

  def bn_rl_conv(x, f, k=1, s=1, p='same'):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(f, k, strides=s, padding=p)(x)
    return x

  def dense_block(tensor, r):
    for _ in range(r):
      x = bn_rl_conv(tensor, 4*f)
      x = bn_rl_conv(x, f, 3)
      tensor = Concatenate()([tensor, x])
    return tensor


  def transition_block(x):
    x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
    x = AvgPool2D(2, strides=2, padding='same')(x)
    return x


  input = Input(img_shape)

  x = Conv2D(64, 7, strides=2, padding='same')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  for r in repetitions:
    d = dense_block(x, r)
    x = transition_block(d)

  x = GlobalAvgPool2D()(d)

  output = Dense(n_classes, activation='softmax')(x)

  model = Model(input, output)
  return model

In [ ]:
# Ajustar os dados de entrada para o formato esperado pela rede
imagesTraining = np.array(imagesTraining).astype('float32')
imagesTesting = np.array(imagesTesting).astype('float32')
imagesTraining /= 255.0
imagesTesting /= 255.0

# Definir o número de classes do problema
num_classes = 10

labelsTraining = np.array(labelsTraining)
labelsTraining = to_categorical(labelsTraining, num_classes)
labelsTesting = np.array(labelsTesting)
labelsTesting = to_categorical(labelsTesting, num_classes)

# Construir o modelo ResNet
input_shape = imagesTraining[0].shape
model = densenet(input_shape, num_classes)

NameError: ignored

# Treinando Modelo

In [ ]:
# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(imagesTraining, labelsTraining, batch_size=32, epochs=10, validation_data=(imagesTesting, labelsTesting))

In [ ]:
# Salvar o modelo
model.save('/content/drive/MyDrive/CNN_SOURCE/Modelos/cnn_modelo_densenet.h5')